In [2]:
# To combine all predictions into a master predictions json file 
import json
from pathlib import Path

# Note - because we don't have all the images, we would have to sort through the exported docs folder
runs_root = Path("runs")
output_file = "predictions_dict_master.json"

# Master predictions list
master_predictions = []

# Loop through files matching the pattern
for json_file in sorted(runs_root.glob("*/predictions_dict_*.json")):
    with open(json_file, "r") as f:
        data = json.load(f)
        if "predictions" in data:
            master_predictions.extend(data["predictions"])  # Concatenate predictions!
        else:
            print(f"{json_file} missing 'predictions' key")

# Write the combined predictions to a new file
with open(output_file, "w") as f:
    json.dump({"predictions": master_predictions}, f, indent=2)

print(f"Combined {len(master_predictions)} predictions into {output_file}")

Combined 0 predictions into predictions_dict_master.json
